In [83]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import Markdown as md
from datetime import date, datetime
%matplotlib inline
#sns.set(rc={"figure.figsize":(20, 20)})

from pandas import DataFrame
from typing import List

2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :

* ○ temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
* ○ dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
* ○ atraso_15 : 1 si dif_min > 15, 0 si no.
* ○ periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I .

In [84]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [85]:
# COLUMNS
FECHAI = 'Fecha-I'
FECHAO = 'Fecha-O'
TEMPORADA_ALTA = 'temporada_alta'
DIF_MIN = 'dif_min'
ATRASO_15 = 'atraso_15'
PERIODO_DIA = 'periodo_dia'
MORNING = 'mañana'
AFTERNOON = 'tarde'
NIGHT = 'noche'


DATE = 'Date'
TIME = 'time'
HOUR = 'hour'
MINUTE = 'minute'

columns = [TEMPORADA_ALTA, DIF_MIN, ATRASO_15, PERIODO_DIA]

In [86]:
df[FECHAI] = pd.to_datetime(df[FECHAI])
df[FECHAO] = pd.to_datetime(df[FECHAO])

In [87]:
df[DATE] = df['Fecha-I'].dt.date
df[TIME] = df['Fecha-I'].dt.time
df[HOUR] = df['Fecha-I'].dt.hour
df[MINUTE] = df['Fecha-I'].dt.minute

In [88]:
def convert_str_to_date(str_date: str) -> datetime:
    """convert string to date format '%Y-%m-%d'

    Args:
        str_date (str): string ex: '2017-01-01'

    Returns:
        datetime: string with datetime type
    """
    return datetime.strptime(str_date, '%Y-%m-%d').date()

def convert_str_to_time(str_date: str) -> datetime:
    """convert string to time format '%H:%M'

    Args:
        str_date (str): string '23:59'

    Returns:
        datetime: string with datetime type
    """
    return datetime.strptime(str_date, '%H:%M').time()

def add_temporada_alta_flag(df: DataFrame, dates: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    df[TEMPORADA_ALTA] = 0
    for date in dates:
        df.loc[(df[DATE] >= date[0]) & (df[DATE]<=date[1]), TEMPORADA_ALTA] = 1
    return df.copy()
def add_periodo_dia_flag(df: DataFrame, times: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    # default
    df[PERIODO_DIA] = NIGHT
    df.loc[(df[TIME] >= times[0][0]) & (df[TIME]<=times[0][1]), PERIODO_DIA] = MORNING
    df.loc[(df[TIME] >= times[1][0]) & (df[TIME]<=times[1][1]), PERIODO_DIA] = AFTERNOON
    return df.copy()

In [89]:
str_winter_start = '2017-12-15'
str_winter_end = '2017-12-31'
str_winter_bis_start = '2017-01-01'
str_winter_bis_end = '2017-03-03'
str_July_start = '2017-07-15'
str_July_end = '2017-07-31'
str_Sept_start = '2017-09-11'
str_Sept_end = '2017-09-30'

str_winter_start = convert_str_to_date(str_winter_start)
str_winter_end = convert_str_to_date(str_winter_end)
str_winter_bis_start = convert_str_to_date(str_winter_bis_start)
str_winter_bis_end = convert_str_to_date(str_winter_bis_end)
str_July_start = convert_str_to_date(str_July_start)
str_July_end = convert_str_to_date(str_July_end)
str_Sept_start = convert_str_to_date(str_Sept_start)
str_Sept_end = convert_str_to_date(str_Sept_end)

dates = [[str_winter_start, str_winter_end],
         [str_winter_bis_start, str_winter_bis_end],
         [str_July_start, str_July_end],
         [str_Sept_start, str_Sept_end]]

In [90]:
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,Date,time,hour,minute
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,2017-01-01,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,2017-01-02,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,2017-01-03,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,2017-01-04,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,2017-01-05,23:30:00,23,30


In [91]:
add_temporada_alta_flag(df, dates)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,Date,time,hour,minute,temporada_alta
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,2017-01-01,23:30:00,23,30,1
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,2017-01-02,23:30:00,23,30,1
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,2017-01-03,23:30:00,23,30,1
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,2017-01-04,23:30:00,23,30,1
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,2017-01-05,23:30:00,23,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,Viernes,I,JetSmart SPA,Santiago,Lima,2017-12-22,14:55:00,14,55,1
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,Lunes,I,JetSmart SPA,Santiago,Lima,2017-12-25,14:55:00,14,55,1
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,Miercoles,I,JetSmart SPA,Santiago,Lima,2017-12-27,14:55:00,14,55,1
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,Viernes,I,JetSmart SPA,Santiago,Lima,2017-12-29,14:55:00,14,55,1


In [92]:
df[DIF_MIN] = df[FECHAO]-df[FECHAI]

In [93]:
df[DIF_MIN] = df['dif_min'].dt.total_seconds()//60
df[ATRASO_15] = 0
df.loc[df[DIF_MIN]>15, ATRASO_15] = 1

In [94]:
# periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59)
#y noche (entre 19:00 y 4:59), en base a Fecha-I


In [95]:
str_mañana_start= '5:00'
str_mañana_end = '11:59'
str_tarde_start = '12:00'
str_tarde_end = '18:59'
# str_noche_start_1 = '19:00'
# str_noche_start_2 = '00:00'
# str_noche_end_1 = '23:59'
# str_noche_end_2 = '4:59'

str_mañana_start = convert_str_to_time(str_mañana_start)
str_mañana_end = convert_str_to_time(str_mañana_end)
str_tarde_start = convert_str_to_time(str_tarde_start)
str_tarde_end = convert_str_to_time(str_tarde_end)
times = [[str_mañana_start, str_mañana_end],
         [str_tarde_start, str_tarde_end],
#         [str_noche_start_1, str_noche_end_1],
#         [str_noche_start_2, str_noche_end_2]
        ]

In [96]:
str_mañana_start

datetime.time(5, 0)

In [97]:
add_periodo_dia_flag(df, times)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,SIGLAORI,SIGLADES,Date,time,hour,minute,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,2017-01-01,23:30:00,23,30,1,3.0,0,noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,2017-01-02,23:30:00,23,30,1,9.0,0,noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,2017-01-03,23:30:00,23,30,1,9.0,0,noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,2017-01-04,23:30:00,23,30,1,3.0,0,noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,2017-01-05,23:30:00,23,30,1,-3.0,0,noche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,2017-12-22,14:55:00,14,55,1,46.0,1,tarde
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,2017-12-25,14:55:00,14,55,1,16.0,1,tarde
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,2017-12-27,14:55:00,14,55,1,40.0,1,tarde
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,2017-12-29,14:55:00,14,55,1,13.0,0,tarde


In [98]:
df.temporada_alta.value_counts()

0    45444
1    22762
Name: temporada_alta, dtype: int64

In [99]:
df.periodo_dia.value_counts()

tarde     25506
mañana    25352
noche     17348
Name: periodo_dia, dtype: int64

In [100]:
df[columns].to_csv('synthetic_features.csv',index= False)

In [101]:
df.sort_values(FECHAI, inplace = True)
df["Early"] = df[DIF_MIN].shift(1)
df['AnteriorEarly'] = 0
df.loc[df["Early"]<0, 'AnteriorEarly'] = 1
df.loc[df['AnteriorEarly'].isna(), 'AnteriorEarly'] = 0
df["Delay"] = df[DIF_MIN].shift(1)
df['AnteriorDelay'] = 0
df.loc[df["Delay"]>0, 'AnteriorDelay'] = 1
del df['Delay']
del df['Early']

In [102]:
dft = df.groupby([FECHAI])[FECHAI].count().rename("col_tmp")
dft = dft.reset_index()
dft["number_of_flights_before"] = dft["col_tmp"].shift(1)
del dft['col_tmp']
df = pd.merge(df,dft)
df.loc[df['number_of_flights_before'].isna(), 'number_of_flights_before'] = 0

In [103]:
dft = df.groupby([FECHAI, 'OPERA'])[FECHAI].count().rename("number_of_flights_same_opera")
dft = dft.reset_index()
dft['number_of_flights_same_opera'] = dft['number_of_flights_same_opera'] -1 
df = pd.merge(df,dft)
df.loc[df['number_of_flights_same_opera'].isna(), 'number_of_flights_same_opera'] = 0

In [104]:
dft.number_of_flights_same_opera.value_counts()

0    55903
1     5529
2      403
3        9
Name: number_of_flights_same_opera, dtype: int64

In [111]:
dft = df.groupby([FECHAI, 'SIGLADES'])[FECHAI].count().rename("number_of_flights_same_dest")
dft = dft.reset_index()
dft['number_of_flights_same_dest'] = dft['number_of_flights_same_dest'] -1 
df = pd.merge(df,dft)
df.loc[df['number_of_flights_same_dest'].isna(), 'number_of_flights_same_dest'] = 0

In [115]:
df.number_of_flights_same_dest.value_counts()

0    66588
1     1618
Name: number_of_flights_same_dest, dtype: int64